<a href="https://colab.research.google.com/github/yyboom/GoogleGemma/blob/master/20241004_GMBC_Gemma_Sprint_korean_real_estate_transaction_advisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U peft
%pip install -U accelerate
%pip install -U trl
%pip install -U datasets
%pip install -U kaggle

In [ ]:
import torch
import re
import os
import io
import pandas as pd
import json
from google.colab import files
from google.colab import userdata
from google.colab import files
from datasets import Dataset, load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline, TrainingArguments
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Apart Deal.csv')

<ipython-input-4-c9ab1da95f4a>:1: DtypeWarning: Columns (6,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/MyDrive/Apart Deal.csv')


In [ ]:
def create_input(row):
    # 부동산 거래 정보를 입력으로 생성
    profile = (
        f"부동산 거래 정보:\n"
        f"- 거래일: {row['거래일']}\n"
        f"- 매물명: {row['법정동']}\n"
        f"- 지번: {row['지번']}\n"
        f"- 전용면적: {row['전용면적']}㎡\n"
    )
    return profile

def create_output(row):
    # 거래 금액 정보를 포함한 출력 생성
    output = (
        f"해당 매물은 {row['거래일']}에 거래되었으며, 전용면적 {row['전용면적']}㎡의 {row['법정동']}입니다. "
        f"지번은 {row['지번']}이고, 거래 금액은 {row['거래금액']}원입니다."
    )
    return output

# 'input' 열 생성
df['input'] = df.apply(create_input, axis=1)

# 'output' 열 생성
df['output'] = df.apply(create_output, axis=1)

# 'input'과 'output'을 결합해 'text' 열 생성
df['text'] = df.apply(lambda row: f"input: {row['input']}\noutput: {row['output']}", axis=1)


In [ ]:
# 결측값이 있는 행 제거
df.dropna(subset=['input', 'output'], inplace=True)

# 데이터프레임 섞기
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
# 학습을 위한 'text' 열 준비
df['text'] = df.apply(lambda row: f"input: {row['input']}\noutput: {row['output']}", axis=1)

In [ ]:
# 입력-출력 쌍 예시 출력
example_row = df.iloc[0]
print("입력:")
print(example_row['input'])
print("\n출력:")
print(example_row['output'])

입력:
부동산 거래 정보:
- 거래일: 5/6/2020 0:00
- 매물명: 진동면 진동리
- 지번: 424
- 전용면적: 59.925㎡


출력:
해당 매물은 5/6/2020 0:00에 거래되었으며, 전용면적 59.925㎡의 진동면 진동리입니다. 지번은 424이고, 거래 금액은 14500원입니다.


In [ ]:
# 입력-출력 쌍 예시 출력
example_row = df.iloc[0]
print("입력:")
print(example_row['input'])
print("\n출력:")
print(example_row['output'])

입력:
부동산 거래 정보:
- 거래일: 5/6/2020 0:00
- 매물명: 진동면 진동리
- 지번: 424
- 전용면적: 59.925㎡


출력:
해당 매물은 5/6/2020 0:00에 거래되었으며, 전용면적 59.925㎡의 진동면 진동리입니다. 지번은 424이고, 거래 금액은 14500원입니다.


In [ ]:
from datasets import Dataset

# 'text' 열에서 결측값이 있는 행 제거
df.dropna(subset=['text'], inplace=True)

# pandas DataFrame에서 Dataset 생성
train_data = Dataset.from_pandas(df[['text']])

# Dataset 확인
print(train_data)


In [ ]:
# 모델 초기화
model_name = "/kaggle/input/gemma/transformers/2b-it/3"  # 모델 경로
compute_dtype = getattr(torch, "float16")  # 계산 데이터 타입 설정

# Bits and Bytes 구성 설정
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4비트 양자화 로드
    bnb_4bit_use_double_quant=False,  # 이중 양자화 사용 여부
    bnb_4bit_quant_type="nf4",  # 4비트 양자화 유형
    bnb_4bit_compute_dtype=compute_dtype,  # 계산 데이터 타입
)

# 모델 불러오기
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,  # 양자화 설정
    torch_dtype=compute_dtype,  # 계산 데이터 타입
    low_cpu_mem_usage=True,  # CPU 메모리 사용량 최소화
)

# 모델 설정 수정
model.config.use_cache = False  # 캐시 사용 비활성화
model.config.pretraining_tp = 1  # 사전 훈련 텐서 병렬화 설정

model.config.hidden_activation = 'gelu_pytorch_tanh'  # 숨겨진 활성화 함수 설정

max_seq_length = 1024  # 최대 시퀀스 길이 설정
tokenizer = AutoTokenizer.from_pretrained(model_name, max_seq_length=max_seq_length)  # 토크나이저 불러오기

In [ ]:
# 모델 파인튜닝 설정
peft_config = LoraConfig(
    lora_alpha=16,  # LoRA의 알파 값
    lora_dropout=0,  # LoRA 드롭아웃 비율
    r=64,  # LoRA의 차원 수
    bias="none",  # 바이어스 처리 방법
    task_type="CAUSAL_LM",  # 작업 유형: CAUSAL_LM (원인적 언어 모델링)
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",  # LoRA를 적용할 모듈
                    "gate_proj", "up_proj", "down_proj"],
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="korean-real-estate-transaction_advisor_model",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=5e-4,
    weight_decay=0.001,
    fp16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
)

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=train_data,
    peft_config=peft_config,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
)

# 학습
trainer.train()

In [ ]:
# 저장
trainer.save_model()
tokenizer.save_pretrained("korean-real-estate-transaction_advisor_model")

from peft import AutoPeftModelForCausalLM

model = AutoPeftModelForCausalLM.from_pretrained(
     "korean-real-estate-transaction_advisor_model",
     torch_dtype=compute_dtype,
     device_map="auto",
)

merged_model = model.merge_and_unload()
merged_model.save_pretrained("./korean-real-estate-transaction_advisor_pretrained", safe_serialization=True, max_shard_size="2GB")
tokenizer.save_pretrained("./korean-real-estate-transaction_advisor_pretrained")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# 사전 훈련된 모델 디렉토리 경로 지정
model_name = "./korean-real-estate-transaction_advisor_pretrained"

# 디바이스 맵 없이 모델 로드
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,  # 데이터 타입 설정
)

# GPU가 사용 가능하면 모델을 GPU로 이동
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 재정 조언 생성 함수 정의
def get_advice(user_profile, model=model, tokenizer=tokenizer):
    prompt = f"input: {user_profile}\noutput:"
    # 입력을 토크나이즈하고 텐서를 모델과 동일한 디바이스로 이동
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,  # 생성할 최대 토큰 수
        no_repeat_ngram_size=2,  # 반복되는 n-그램 방지
        early_stopping=True,  # 조기 종료 설정
    )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)  # 생성된 응답 디코딩
    return response.split('output:')[-1].strip()  # 'output:' 이후의 내용을 반환

# 예시 사용법
example_input = df.iloc[0]['input']
print("거래 정보가 궁금한 동네 입력:")
print(example_input)
print("\n생성된 조언:")
print(get_advice(example_input))
